#### **Extract, Transform and Load (ETL) and Data Modelling in Python** 
- **The following scripts demonstrate the ETL and data modelling process:**
1. Extracting data: Read in the datasets named Orders and Details using pandas
2. Initial Exploration: checked the structure and key attributes of the two dataframes before joining them
3. Column Standardization: Renamed columns for consistency and better readability
4. Data modelling: Created the customer dimension dataframe and added a surrogate key, created the product dimension dataframe and added a surrogate key, created the date dimension dataframe and added a surrogate key, merge the five dataframes to create the orders fact dataframe and added a surrogate key
5. Connecting to SQL Server and pushing the dataframes created: Connected to a database named OnlineSales created in SQL Server and pushed the orders fact, the customer dimension, the product dimension, and the date dimension dataframes to the database

##### **Data source** (Orders and Details csv files): https://www.kaggle.com/datasets/samruddhi4040/online-sales-data?select=Orders.csv


In [1]:
# Import libararies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import urllib
from urllib.parse import quote_plus
import pypyodbc as odbc  

In [ ]:
# Read in the two tables, Orders and Details
Orders_df = pd.read_csv(r'Online Sales Data\Orders.csv')
Details_df = pd.read_csv(r'Online Sales Data\Details.csv')

In [8]:
# Check the first five rows of the Orders dataframe
Orders_df.head()

,Order ID,Order Date,CustomerName,State,City
0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura
1,B-25993,03-02-2018,Madhav,Delhi,Delhi
2,B-25973,24-01-2018,Madan Mohan,Uttar Pradesh,Mathura
3,B-25923,27-12-2018,Gopal,Maharashtra,Mumbai
4,B-25757,21-08-2018,Vishakha,Madhya Pradesh,Indore


In [ ]:
# Show the structural summary of the Orders dataframe
Orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [ ]:
# Show the descriptive statistics of the Order dataframe
Orders_df.describe(include='all')

,Order ID,Order Date,CustomerName,State,City
count,500,500,500,500,500
unique,500,307,336,19,25
top,B-26055,24-11-2018,Shreya,Maharashtra,Indore
freq,1,7,6,94,71


In [12]:
# Check the first five rows of the Details dataframe
Details_df.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode
0,B-25681,1096,658,7,Electronics,Electronic Games,COD
1,B-26055,5729,64,14,Furniture,Chairs,EMI
2,B-25955,2927,146,8,Furniture,Bookcases,EMI
3,B-26093,2847,712,8,Electronics,Printers,Credit Card
4,B-25602,2617,1151,4,Electronics,Phones,Credit Card


In [ ]:
# Show the structural summary of the Details dataframe
Details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      1500 non-null   object
 1   Amount        1500 non-null   int64 
 2   Profit        1500 non-null   int64 
 3   Quantity      1500 non-null   int64 
 4   Category      1500 non-null   object
 5   Sub-Category  1500 non-null   object
 6   PaymentMode   1500 non-null   object
dtypes: int64(3), object(4)
memory usage: 82.2+ KB


In [13]:
# Show the descriptive statistics of the Details dataframe
Details_df.describe(include='all')

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode
count,1500,1500.000000,1500.00000,1500.000000,1500,1500,1500
unique,500,NaN,NaN,NaN,3,17,5
top,B-25656,NaN,NaN,NaN,Clothing,Saree,COD
freq,12,NaN,NaN,NaN,949,211,684
mean,NaN,291.847333,24.64200,3.743333,NaN,NaN,NaN
std,NaN,461.924620,168.55881,2.184942,NaN,NaN,NaN
min,NaN,4.000000,-1981.00000,1.000000,NaN,NaN,NaN
25%,NaN,47.750000,-12.00000,2.000000,NaN,NaN,NaN
50%,NaN,122.000000,8.00000,3.000000,NaN,NaN,NaN
75%,NaN,326.250000,38.00000,5.000000,NaN,NaN,NaN


In [ ]:
# Join the two dataframes on Order ID
merged_Orders_Details = Orders_df.merge(Details_df, on='Order ID', how='left')
merged_Orders_Details

,Order ID,Order Date,CustomerName,State,City,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode
0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,5729,64,14,Furniture,Chairs,EMI
1,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,671,114,9,Electronics,Phones,Credit Card
2,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,443,11,1,Clothing,Saree,COD
3,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,57,7,2,Clothing,Shirt,UPI
4,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,227,48,5,Clothing,Stole,COD
...,...,...,...,...,...,...,...,...,...,...,...
1495,B-25742,03-08-2018,Ashwin,Goa,Goa,11,-8,2,Clothing,Skirt,UPI
1496,B-26088,26-03-2018,Bhavna,Sikkim,Gangtok,11,5,2,Clothing,Hankerchief,UPI
1497,B-25707,01-07-2018,Shivani,Maharashtra,Mumbai,8,-6,1,Clothing,Stole,COD
1498,B-25758,22-08-2018,Shubham,Himachal Pradesh,Simla,8,-2,1,Clothing,Stole,COD


In [17]:
# Rename columns
merged_Orders_Details.rename(columns={'Order ID': 'OrderID', 'Order Date': 'OrderDate', 'Sub-Category': 'SubCategory'}, inplace=True)

merged_Orders_Details.head()

,OrderID,OrderDate,CustomerName,State,City,Amount,Profit,Quantity,Category,SubCategory,PaymentMode
0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,5729,64,14,Furniture,Chairs,EMI
1,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,671,114,9,Electronics,Phones,Credit Card
2,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,443,11,1,Clothing,Saree,COD
3,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,57,7,2,Clothing,Shirt,UPI
4,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura,227,48,5,Clothing,Stole,COD


In [ ]:
# Create customer dimension dataframe
dim_customer = (
    merged_Orders_Details[['CustomerName', 'State', 'City']]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={'index': 'CustomerKey'})
)

# Add a surrogate key starting from 1
dim_customer['CustomerKey'] += 1  

In [ ]:
# Show the dim_customer dataframe created
dim_customer

,CustomerKey,CustomerName,State,City
0,1,Harivansh,Uttar Pradesh,Mathura
1,2,Madhav,Delhi,Delhi
2,3,Madan Mohan,Uttar Pradesh,Mathura
3,4,Gopal,Maharashtra,Mumbai
4,5,Vishakha,Madhya Pradesh,Indore
...,...,...,...,...
400,401,Hemangi,Delhi,Delhi
401,402,Dinesh,Tamil Nadu,Chennai
402,403,Ashwin,Goa,Goa
403,404,Shivani,Maharashtra,Mumbai


In [ ]:
# Create product dimension dataframe
dim_product = (
    merged_Orders_Details[['Category', 'SubCategory']]
    .drop_duplicates()
    .reset_index(drop=True)
    .reset_index()
    .rename(columns={'index': 'ProductKey'})
)

# Add a surrogate key starting from 1
dim_product['ProductKey'] += 1

In [ ]:
# Show the dim_product dataframe created
dim_product

,ProductKey,Category,SubCategory
0,1,Furniture,Chairs
1,2,Electronics,Phones
2,3,Clothing,Saree
3,4,Clothing,Shirt
4,5,Clothing,Stole
5,6,Clothing,T-shirt
6,7,Electronics,Printers
7,8,Furniture,Bookcases
8,9,Furniture,Furnishings
9,10,Furniture,Tables


In [ ]:
# Create date dimension dataframe

# Create date range
start_date = merged_Orders_Details['OrderDate'].min()
end_date = merged_Orders_Details['OrderDate'].max()
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Build dim_date dataframe
dim_date = pd.DataFrame(dates, columns=['Date'])
dim_date['Year'] = dim_date['Date'].dt.year
dim_date['Quarter'] = dim_date['Date'].dt.quarter
dim_date['Month'] = dim_date['Date'].dt.month
dim_date['Day'] = dim_date['Date'].dt.day
dim_date['DayOfWeek'] = dim_date['Date'].dt.dayofweek + 1   # Monday=1, Sunday=7
dim_date['DayName'] = dim_date['Date'].dt.day_name()
dim_date['IsWeekend'] = dim_date['DayOfWeek'].isin([6, 7]).astype(int)

# Add a surrogate key
dim_date['DateKey'] = dim_date['Date'].dt.strftime('%Y%m%d').astype(int)

# Change date format to dd/mm/yyyy
dim_date['Date'] = dim_date['Date'].dt.strftime('%d-%m-%Y')

# Reorganize columns
dim_date = dim_date[['DateKey', 'Date', 'Year', 'Quarter', 'Month', 'Day', 'DayOfWeek', 'DayName', 'IsWeekend']]


In [ ]:
# Show the dim_date dataframe created
dim_date

,DateKey,Date,Year,Quarter,Month,Day,DayOfWeek,DayName,IsWeekend
0,20180101,01-01-2018,2018,1,1,1,1,Monday,0
1,20180102,02-01-2018,2018,1,1,2,2,Tuesday,0
2,20180103,03-01-2018,2018,1,1,3,3,Wednesday,0
3,20180104,04-01-2018,2018,1,1,4,4,Thursday,0
4,20180105,05-01-2018,2018,1,1,5,5,Friday,0
...,...,...,...,...,...,...,...,...,...
360,20181227,27-12-2018,2018,4,12,27,4,Thursday,0
361,20181228,28-12-2018,2018,4,12,28,5,Friday,0
362,20181229,29-12-2018,2018,4,12,29,6,Saturday,1
363,20181230,30-12-2018,2018,4,12,30,7,Sunday,1


In [25]:
# Show columns of the five tables: Orders, Details, customer_dim, product_dim and date_dim
print(f'merged_Orders_Details columns: {merged_Orders_Details.columns}')
print(f'dim_customer columns: {dim_customer.columns}')
print(f'dim_product columns: {dim_product.columns}')
print(f'dim_date columns: {dim_date.columns}')

merged_Orders_Details columns: Index(['OrderID', 'OrderDate', 'CustomerName', 'State', 'City', 'Amount',
       'Profit', 'Quantity', 'Category', 'SubCategory', 'PaymentMode'],
      dtype='object')
dim_customer columns: Index(['CustomerKey', 'CustomerName', 'State', 'City'], dtype='object')
dim_product columns: Index(['ProductKey', 'Category', 'SubCategory'], dtype='object')
dim_date columns: Index(['DateKey', 'Date', 'Year', 'Quarter', 'Month', 'Day', 'DayOfWeek',
       'DayName', 'IsWeekend'],
      dtype='object')


In [ ]:
# Merge the five tables to create orders dataframe
merged_table = merged_Orders_Details.merge(
    dim_customer, on=['CustomerName', 'State', 'City'], how='left').merge(
        dim_product, on=['Category', 'SubCategory'], how='left').merge(
            dim_date[['Date', 'DateKey']], left_on='OrderDate', right_on='Date', how='left')

# Create a surrogate key starting from 1
merged_table = merged_table.reset_index(drop=True).reset_index().rename(columns={'index': 'OrdersKey'})
merged_table['OrdersKey'] += 1

# Select columns for the fact_orders dataframe
fact_orders = merged_table[[
    'OrdersKey', 'CustomerKey', 'ProductKey', 'DateKey', 'OrderID', 'OrderDate', 
    'Amount', 'Profit', 'Quantity', 'PaymentMode'
]]

In [ ]:
# Show the fact_orders dataframe created
fact_orders

,OrdersKey,CustomerKey,ProductKey,DateKey,OrderID,OrderDate,Amount,Profit,Quantity,PaymentMode
0,1,1,1,20180310,B-26055,10-03-2018,5729,64,14,EMI
1,2,1,2,20180310,B-26055,10-03-2018,671,114,9,Credit Card
2,3,1,3,20180310,B-26055,10-03-2018,443,11,1,COD
3,4,1,4,20180310,B-26055,10-03-2018,57,7,2,UPI
4,5,1,5,20180310,B-26055,10-03-2018,227,48,5,COD
...,...,...,...,...,...,...,...,...,...,...
1495,1496,403,15,20180803,B-25742,03-08-2018,11,-8,2,UPI
1496,1497,286,11,20180326,B-26088,26-03-2018,11,5,2,UPI
1497,1498,404,5,20180701,B-25707,01-07-2018,8,-6,1,COD
1498,1499,405,5,20180822,B-25758,22-08-2018,8,-2,1,COD


In [30]:
# Show the structural summary of the fact_orders dataframe
fact_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   OrdersKey    1500 non-null   int64 
 1   CustomerKey  1500 non-null   int64 
 2   ProductKey   1500 non-null   int64 
 3   DateKey      1500 non-null   int64 
 4   OrderID      1500 non-null   object
 5   OrderDate    1500 non-null   object
 6   Amount       1500 non-null   int64 
 7   Profit       1500 non-null   int64 
 8   Quantity     1500 non-null   int64 
 9   PaymentMode  1500 non-null   object
dtypes: int64(7), object(3)
memory usage: 117.3+ KB


In [34]:
# Show the descriptive statistics of the fact_orders dataframe
fact_orders.describe()

,OrdersKey,CustomerKey,ProductKey,DateKey,Amount,Profit,Quantity
count,1500.000000,1500.000000,1500.00000,1.500000e+03,1500.000000,1500.00000,1500.000000
mean,750.500000,152.584667,7.92600,2.018061e+07,291.847333,24.64200,3.743333
std,433.157015,103.531467,4.72194,3.680470e+02,461.924620,168.55881,2.184942
min,1.000000,1.000000,1.00000,2.018010e+07,4.000000,-1981.00000,1.000000
25%,375.750000,67.000000,4.00000,2.018031e+07,47.750000,-12.00000,2.000000
50%,750.500000,138.000000,7.00000,2.018053e+07,122.000000,8.00000,3.000000
75%,1125.250000,225.250000,11.00000,2.018100e+07,326.250000,38.00000,5.000000
max,1500.000000,405.000000,17.00000,2.018123e+07,5729.000000,1864.00000,14.000000


In [ ]:
# Create a database named OnlineSales in SQL Server and connect to the database
DRIVER_NAME = 'SQL SERVER'
SERVER_NAME = r'MSI\SQLEXPRESS'
DATABASE_NAME = 'OnlineSales'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};UID=[UserName];PWD=[Password]"  
)   # Remove/replace UserName (UID) and Password (PWD) if needed

engine = sqlalchemy.create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Push the tables to the SQL Server database
"""WARNING: When using "if_exists='replace'", the tables will be replaced if they already exist in database, 
and the data model created earlier will be deleted. So use it with caution."""
fact_orders.to_sql('fact_orders', engine, if_exists='replace', index=False)                              
dim_customer.to_sql('dim_customer', engine, if_exists='replace', index=False)   
dim_product.to_sql('dim_product', engine, if_exists='replace', index=False)       
dim_date.to_sql('dim_date', engine, if_exists='replace', index=False)          

132